Objectifs du notebook :

- Nettoyer les données pour obtenir un seul fichier contenant l'échantillon qui sera utilisée.
- Vérifier la présence de valeurs manquantes ou erronées.
- S'assurer de réunir uniquement les données nécessaires pour la base de données.
- Vérifier le typage des données correspondant à la base de données prévue.

[Lien DrawDB](https://drawdb.vercel.app/editor?shareId=e6c18b8ae53063fa1dfa9cc8a849605f)

In [81]:
import matplotlib.pyplot as plt
from rich import print_json, print
from tqdm import tqdm
import pandas as pd
import glob
import os

Récupération de la liste des fichiers du dataset disponible

In [82]:
def get_csv_list(part_number):
    csv_files = []
    if type(part_number) == list:
        for part in part_number:
            csv_files += glob.glob(f"./data/part_{part}/*.csv")
    else:
        csv_files = glob.glob(f"./data/part_{part_number}/*.csv")
    return csv_files

Création des DataFrames finaux

In [83]:
x_post_df = pd.DataFrame({
    "post_id": pd.Series(dtype="int"),
    "user_id": pd.Series(dtype="int"),
    "lang": pd.Series(dtype="str"),
    "text": pd.Series(dtype="str"),
    "date": pd.Series(dtype="datetime64[ns]"),
    "like_count": pd.Series(dtype="int"),
    "reply_count": pd.Series(dtype="int"),
    "retweet_count": pd.Series(dtype="int"),
    "quote_count": pd.Series(dtype="int"),
})

print(x_post_df.dtypes)
x_post_df.to_parquet("x_post.parquet", index=False)

x_user_df = pd.DataFrame({
    "user_id": pd.Series(dtype="int"),
    "name": pd.Series(dtype="str"),
    "bio": pd.Series(dtype="str"),
    "followers_count": pd.Series(dtype="int"),
    "follows_count": pd.Series(dtype="int")
})

print(x_user_df.dtypes)
x_user_df.to_parquet("x_user.parquet", index=False)

post_id                   int64
user_id                   int64
lang                     object
text                     object
date             datetime64[ns]
like_count                int64
reply_count               int64
retweet_count             int64
quote_count               int64
dtype: object

user_id             int64
name               object
bio                object
followers_count     int64
follows_count       int64
dtype: object

Clean function
- input : chemin du csv a ajouter, le dataframe original
- output : le dataframe avec les nouvelles lignes

In [84]:
def clean_file(path, x_post_df, x_user_df):
    current_df = pd.read_csv(path)
    for i, row in tqdm(current_df.iterrows(), total=len(current_df), desc=f"Processing {os.path.basename(path)}"):
        try:
            
            # Vérification des données
            # Nous voulons :
            # - Les lignes sans valeurs nulles
            # - Les tweets en français
            # - Sans retweets ni citation de tweets.
            required_columns = ["id", "lang", "text", "date", "likeCount", "replyCount", "retweetCount", "quoteCount", "user"]
            if any(col not in row or pd.isna(row[col]) for col in required_columns):
                continue
            if row["lang"] != "fr": # Choisir language voulu
                continue
            if row["quotedTweet"] == True:
                continue
            if row["retweetedTweet"] == True:
                continue
            
            ######### USER PART #########
            import datetime # Import datetime pour évaluer directement le dictionnaire "user"

            user_dict = eval(row["user"]) # Evaluer la ligne
            user_id = int(user_dict["id"])
            user_name = user_dict["username"]
            user_bio = user_dict["rawDescription"]
            user_followers_count = int(user_dict["followersCount"])
            user_follows_count = int(user_dict["friendsCount"])

            ######### POST PART #########
        
            post_id = int(row["id"])
            lang = str(row["lang"])
            text = str(row["text"])
            date = str(row["date"])
            like_count = int(row["likeCount"])
            reply_count = int(row["replyCount"])
            retweet_count = int(row["retweetCount"])
            quote_count = int(row["quoteCount"])
            
            ######### AJOUT PART #########

            # Vérification si le tweet existe déjà dans le DataFrame
            if x_post_df["post_id"].eq(post_id).any():
                continue
            else:
                new_row_tweet = pd.DataFrame({
                    "post_id": [post_id],
                    "user_id": [user_id],
                    "lang": [lang],
                    "text": [text],
                    "date": [pd.to_datetime(date)],
                    "like_count": [like_count],
                    "reply_count": [reply_count],
                    "retweet_count": [retweet_count],
                    "quote_count": [quote_count]
                })
                x_post_df = pd.concat([x_post_df, new_row_tweet], ignore_index=True)

            # Vérification si l'utilisateur existe déjà dans le DataFrame
            if x_user_df["user_id"].eq(user_id).any():
                continue
            else:
                new_row_user = pd.DataFrame({
                    "user_id": [user_id],
                    "name": [user_name],
                    "bio": [user_bio],
                    "followers_count": [user_followers_count],
                    "follows_count": [user_follows_count]
                })
                x_user_df = pd.concat([x_user_df, new_row_user], ignore_index=True)


            


        except Exception as e:
            print(f"Error processing row {i} from file {os.path.basename(path)}: {str(e)}")
            continue
    
    return (x_post_df, x_user_df)
        

Cellule a utiliser pour actualiser x_post.parquet

In [85]:
x_post_df = pd.read_parquet("x_post.parquet")
x_user_df = pd.read_parquet("x_user.parquet")

for path in get_csv_list([1]): # Ici mettre une liste ou un numéro de partie du dataset. Pour la version complète mettre [i for i in range(1, 23)]
    x_post_df, x_user_df = clean_file(path, x_post_df, x_user_df)

# x_post confirmation
print("[bold yellow]x_post_df describe:[/bold yellow]")
print(x_post_df.describe())
print("[bold yellow]x_post_df head(5):[/bold yellow]")
print(x_post_df.head())
x_post_df.to_parquet("x_post.parquet", index=False)
print("[bold yellow]Sucessfuly saved to x_post.parquet[/bold yellow]")
# x_user confirmation
print("[bold yellow]x_user_df describe:[/bold yellow]")
print(x_user_df.describe())
print("[bold yellow]x_user_df head(5):[/bold yellow]")
print(x_user_df.head())
x_user_df.to_parquet("x_user.parquet", index=False)
print("[bold yellow]Sucessfuly saved to x_user.parquet[/bold yellow]")

Processing may_july_chunk_11.csv: 100%|██████████| 50000/50000 [00:03<00:00, 15488.91it/s]


x_post_df describe:

post_id       user_id                           date   like_count  \
count  4.931000e+03  4.931000e+03                           4931  4931.000000   
mean   1.801254e+18  9.926936e+17  2024-06-12 23:07:08.554045696    12.640641   
min    1.800679e+18  7.603520e+05            2024-06-12 00:00:00     0.000000   
25%    1.800955e+18  2.362225e+09            2024-06-12 00:00:00     0.000000   
50%    1.801280e+18  1.328145e+18            2024-06-13 00:00:00     0.000000   
75%    1.801579e+18  1.585781e+18            2024-06-14 00:00:00     1.000000   
max    1.801766e+18  1.801654e+18            2024-06-14 00:00:00  4702.000000   
std    3.206305e+14  7.057328e+17                            NaN   138.843174   

       reply_count  retweet_count  quote_count  
count  4931.000000    4931.000000  4931.000000  
mean      1.456702       4.275806     0.908335  
min       0.000000       0.000000     0.000000  
25%       0.000000       0.000000     0.000000  
50%       0.000000       0.000000     0.000000  
75%       1.000000       0.000000     0.000000  
max     557.000000    1887.000000   604.000000  
std      13.465933      54.479765    19.025810

x_post_df head(5):

post_id              user_id lang  \
0  1801761052310310976  1181549520385122304   fr   
1  1801760646603669666  1521599300656107523   fr   
2  1801760474058428922  1335615761365217288   fr   
3  1801760036869300601  1710461091157843975   fr   
4  1801759887405240335  1499768906973593600   fr   

                                                text       date  like_count  \
0  @benoitm_mtl le pape est pas vite vite ,les ga... 2024-06-14           0   
1                  @JdeMontreal Parlons de Biden !!! 2024-06-14           0   
2    @EscadronKoukou Des vrais partisans de Biden 😂😂 2024-06-14          12   
3  @leclerctaillage @rbrtcormier Qui tu propose ?... 2024-06-14           1   
4  @TThierryn @richardhetu Vrai ou pas que c’est ... 2024-06-14           0   

   reply_count  retweet_count  quote_count  
0            0              0            0  
1            0              0            0  
2            1              0            0  
3            1              0            0  
4            0              0            0

...

x_user_df describe:

user_id  followers_count  follows_count
count  3.560000e+03     3.560000e+03    3560.000000
mean   1.001462e+18     1.731395e+04    1486.949719
std    7.054360e+17     1.874167e+05    4835.138097
min    7.603520e+05     0.000000e+00       0.000000
25%    2.526108e+09     1.200000e+02     124.750000
50%    1.334508e+18     3.825000e+02     408.000000
75%    1.588789e+18     1.628250e+03    1234.750000
max    1.801654e+18     5.269112e+06  135762.000000

x_user_df head(5):

user_id                 name  \
0  1181549520385122304                leflu   
1  1521599300656107523  Monsieur Blacky boy   
2  1335615761365217288            Ennio Red   
3  1710461091157843975        user016777v01   
4  1499768906973593600                  FB.   

                                                 bio  followers_count  \
0            jecrit mal mais je vous aime quand meme              327   
1                                                                 269   
2  Love to triggers leftist liberals .i am Conser...            12169   
3                                                                 212   
4   Nouveau et amélioré. Toujours même mauvais goût!              501   

   follows_count  
0            291  
1            178  
2           9901  
3            146  
4            363

Sucessfuly saved to x_user.parquet